# dnaMethyAge
Link: https://github.com/yiluyucheng/dnaMethyAge
## Installing Packages
install.packages("devtools")

## Install from Github

In [50]:
devtools::install_github("yiluyucheng/dnaMethyAge")

# Load the required packages
library('dnaMethyAge')
library(dplyr)
library(purrr)
library(tidyverse)

#Getting help
help(methyAge)

Skipping install of 'dnaMethyAge' from a github remote, the SHA1 (534ac02c) has not changed since last install.
  Use `force = TRUE` to force installation



methyAge              package:dnaMethyAge              R Documentation

_P_r_e_d_i_c_t _e_p_i_g_e_n_e_t_i_c _a_g_e _f_r_o_m _D_N_A _m_e_t_h_y_l_a_t_i_o_n _d_a_t_a

_D_e_s_c_r_i_p_t_i_o_n:

     Predict epigenetic age from DNA methylation data

_U_s_a_g_e:

     methyAge(
       betas,
       clock = "HorvathS2013",
       age_info = NA,
       fit_method = "Linear",
       do_plot = TRUE,
       inputation = TRUE,
       simple_mode = FALSE,
       species = "Homo sapiens",
       MM_array = FALSE,
       use_cores = detectCores()
     )
     
_A_r_g_u_m_e_n_t_s:

   betas: A dataframe with column names as sample ID and row names as
          probe name, beta = M / (M + U + 100).

   clock: Default: 'HorvathS2013', define the clock name to use,
          currently supported clocks are 'HannumG2013', 'HorvathS2013',
          'LevineM2018', and 'ZhangQ2019'.

age_info: Default: NA, in order to calculate the age accelerat

## 1.GSE90124
### 1.1.Copy and Extracting the data

In [4]:
#Copy and unzip the data
GEOID           = "GSE90124"
GSE90124        = paste("/mnt/NAS_PROJECT/vol_Phucteam/CONGNGUYEN/pipeline/Biological_age/Datasets/GEO/",GEOID,"_series_matrix.txt.gz",sep="")
wkdr            = paste("/mnt/NAS_PROJECT/vol_Phucteam/CONGNGUYEN/pipeline/Biological_age/dnaMethyAge_Multi-model/",GEOID,sep="")

# print(wkdr)
# print(GSE90124)

system(paste("cp ",GSE90124," ", wkdr,sep=""),intern=TRUE)
system(paste("gunzip ", wkdr, "/*.gz",sep=""),intern=TRUE)

character(0)

Warning message in system(paste("gunzip ", wkdr, "/*.gz", sep = ""), intern = TRUE):
“running command 'gunzip /mnt/NAS_PROJECT/vol_Phucteam/CONGNGUYEN/pipeline/Biological_age/dnaMethyAge_Multi-model/GSE90124/*.gz' had status 2”


character(0)
attr(,"status")
[1] 2
attr(,"errmsg")
[1] "Resource temporarily unavailable"

### 1.2.Extract the information inside the GSE90124

In [5]:
GSE_file = paste(wkdr,"/",GEOID,"_series_matrix.txt",sep="")

system(paste('grep -E "Series|Sample" ',GSE_file,' > ', wkdr, '/',GEOID,'_metadata',sep=""),intern=TRUE)
system(paste('grep -E -v "Series|Sample|series_matrix" ',GSE_file,' > ', wkdr, '/',GEOID,'_betavalue',sep=""),intern=TRUE)

## Directly do it in the beta_value file for quickly test
betavalue = paste(wkdr, '/',GEOID,'_betavalue',sep="")
betavalue_converted = paste(wkdr, '/',GEOID,'_betavalue_converted',sep="")

system(paste('cat ', betavalue,' | tr "\t" "," | sed \'s/"//g\' > ',betavalue_converted),intern=TRUE)

character(0)

character(0)

character(0)

### Run dnaMethyAge Multi-model on the processed data

In [6]:
## Read the beta value file, Because of the time consumption, this line is run separately (13 minutes)
betas = read.csv(paste(wkdr, '/',GEOID,'_betavalue_converted',sep=""),sep=",")

In [7]:
#Set rownames to be the column ID_REF
rownames(betas) = betas$ID_REF

betas_edited <- betas[ , !(names(betas) %in% 'ID_REF')]

print(head(betas_edited))
print(dim(betas_edited))

           GSM2398638 GSM2398639 GSM2398640 GSM2398641 GSM2398642 GSM2398643
cg00000029  0.2416912  0.2455073  0.1841826  0.2566657  0.2267513  0.2490866
cg00000108  0.9667960  0.9675106  0.9678669  0.9558124  0.9750767  0.9648535
cg00000109  0.8767329  0.8199716  0.8660765  0.8980659  0.8430447  0.8917882
cg00000165  0.2590197  0.1797168  0.2352640  0.2229735  0.1753632  0.2208803
cg00000236  0.8977611  0.8735031  0.8627190  0.9071935  0.8774745  0.8886844
cg00000289  0.6712644  0.6569902  0.7293920  0.8005106  0.6682935  0.7265629
           GSM2398644 GSM2398645 GSM2398646 GSM2398647 GSM2398648 GSM2398649
cg00000029  0.2682283  0.2198363  0.2323712  0.1859844  0.1777726  0.1798672
cg00000108  0.9692669  0.9784556  0.9699124  0.9773629  0.9807906  0.9765199
cg00000109  0.8656042  0.8760436  0.8567279  0.8848552  0.8647298  0.9053908
cg00000165  0.1918849  0.2334030  0.1744057  0.1959975  0.2224219  0.1989373
cg00000236  0.9009682  0.8959790  0.8803353  0.8699794  0.9008277  0.8888202

In [8]:
info = read.csv(paste(wkdr,'/',GEOID,'_metadata_extracted',sep=""),sep="\t")
print(head(info))

      Sample   Age Tissue Gender  BMI Smoking.status                   Twin
1 GSM2398638 42.69   Skin Female 18.7              2  dizygotic_twinpair_60
2 GSM2398639 67.75   Skin Female 35.9              0 monozygotic_twinpair_3
3 GSM2398640 57.59   Skin Female 33.0              2  dizygotic_twinpair_22
4 GSM2398641 49.36   Skin Female 28.5              0   dizygotic_twinpair_6
5 GSM2398642 66.45   Skin Female 28.3              2           singleton_34
6 GSM2398643 75.63   Skin Female 24.8              2           singleton_49


## Run all the possible Clock

In [9]:
logfile = paste(wkdr, '/',GEOID,'_dnaMethyAge_Multi-model.log',sep="")
print(logfile)

for (clock in availableClock()) {
    print(paste('- Processing: ', clock, sep=""))
    message01 = paste('- Processing: ', clock, sep="")
    write(message01, file = logfile, append = TRUE)

    clock_name = clock
    pdf(paste(wkdr, '/Results/', clock_name, '.pdf', sep=""), width=4.3, height=6)
    
    # Use tryCatch to handle errors
    tryCatch({
        clock_result = methyAge(betas_edited, clock=clock_name, age_info=info, fit_method='Linear', do_plot=TRUE)

        #Rename the column "mAge" to "mAge_<clock>"
        colnames(clock_result)[colnames(clock_result) == "mAge"] = paste("mAge_", clock_name, sep="")

        #Rename the column "Age_Acceleration" to "AgeAcc_<clock>"
        colnames(clock_result)[colnames(clock_result) == "Age_Acceleration"] = paste("AgeAcc_", clock_name, sep="")

        #Write to a file
        write.csv(clock_result, paste(wkdr, '/Results/', clock_name, '.csv', sep=""), row.names=FALSE)
        
    }, error = function(e) {
        print(paste("Error in processing: ", clock, ". Skipping to next item.", sep=""))

        message_error = paste("Error in processing: ", clock, ". Skipping to next item.", sep="")
        write(message_error, file = logfile, append = TRUE)

        return()
    })

    message02 = paste('- Done: ', clock, sep="")
    write(message02, file = logfile, append = TRUE)
    
    dev.off()
}


[1] "/mnt/NAS_PROJECT/vol_Phucteam/CONGNGUYEN/pipeline/Biological_age/dnaMethyAge_Multi-model/GSE90124/GSE90124_dnaMethyAge_Multi-model.log"


To understand what do these clocks represent for, please refer to 'https://github.com/yiluyucheng/dnaMethyAge'.



[1] "- Processing: HannumG2013"


Age acceleration is calculated as the residual resulting from a linear regression model which DNAm age is regressed on chronological age.



[1] "- Processing: HorvathS2013"


...

Converting 450k/EPIC to the mammalian array...

Age acceleration is calculated as the residual resulting from a linear regression model which DNAm age is regressed on chronological age.



[1] "- Processing: LuA2023p2"


Converting 450k/EPIC to the mammalian array...

Age acceleration is calculated as the residual resulting from a linear regression model which DNAm age is regressed on chronological age.



[1] "- Processing: LuA2023p3"


Converting 450k/EPIC to the mammalian array...

Age acceleration is calculated as the residual resulting from a linear regression model which DNAm age is regressed on chronological age.



### Merging the results

In [22]:
# Creating list of dataframes:
dfs = list.files(paste(wkdr, '/Results/', sep=""), pattern="*.csv", full.names=TRUE)

# Initialize merged_df as the first dataframe
df1 = read.csv(dfs[[1]])
merged_df <- df1

# Loop over the rest of the dataframes and merge them
for(i in 2:length(dfs)) {
  df = read.csv(dfs[[i]])
  merged_df <- merge(merged_df, df, by = intersect(names(merged_df), names(df)))
}

# Sort alpha order of the column Twin
merged_df <- merged_df[order(merged_df$Twin),]

# Print the merged dataframe
print(merged_df)

# Save the file
write.csv(merged_df, paste(wkdr, '/Results/',GEOID,'_Merged_Multi_Clock.tsv', sep=""),sep="\t", row.names=FALSE)

        Sample   Age Tissue Gender  BMI Smoking.status                    Twin
70  GSM2398707 65.14   Skin Female 28.1              0    dizygotic_twinpair_1
170 GSM2398807 65.14   Skin Female 25.5              0    dizygotic_twinpair_1
317 GSM2398954 46.02   Skin Female 31.6              0   dizygotic_twinpair_10
238 GSM2398875 50.62   Skin Female 21.4              0   dizygotic_twinpair_11
313 GSM2398950 50.62   Skin Female 23.0              2   dizygotic_twinpair_11
126 GSM2398763 42.87   Skin Female 25.9              0   dizygotic_twinpair_12
131 GSM2398768 42.87   Skin Female 29.9              0   dizygotic_twinpair_12
143 GSM2398780 56.14   Skin Female 30.4              2   dizygotic_twinpair_13
312 GSM2398949 56.14   Skin Female 26.6              1   dizygotic_twinpair_13
158 GSM2398795 63.50   Skin Female 26.4              0   dizygotic_twinpair_14
296 GSM2398933 63.50   Skin Female 21.2              2   dizygotic_twinpair_14
223 GSM2398860 63.91   Skin Female 26.2             

Warning message in write.csv(merged_df, paste(wkdr, "/Results/", GEOID, "_Merged_Multi_Clock.tsv", :
“attempt to set 'sep' ignored”


## Handle Multi-Clock data

In [53]:
multi_clock_result = read.csv(paste(wkdr, '/Results/',GEOID,'_Merged_Multi_Clock.tsv', sep=""),sep=",")

# singleton if Twin column contains "singleton_"
singleton = multi_clock_result[grep("singleton_", multi_clock_result$Twin),]
print(dim(singleton))

# twinpair if Twin column contains "*_twinpair_"
twinpair = multi_clock_result[grep("_twinpair_", multi_clock_result$Twin),]

# Only take the records with have at least 2 individuals for each twin (dizygotic_twinpair/monozygotic_twinpair) 
# Define the function
filter_rows <- function(df) {
  df %>%
    group_by(Twin) %>%
    filter(n() >= 2) %>%
    ungroup()
}

twinpair = filter_rows(twinpair)

print(dim(twinpair))


[1] 144  47


[1] 176  47


In [56]:
# Function to calculate RMSE
rmse <- function(actual, predicted) {
  sqrt(mean((actual - predicted)^2))
}

# Get the predictor columns (those that start with 'mAge_')
predictor_cols <- grep("^mAge_", names(twinpair), value = TRUE)
print(predictor_cols)

# Calculate RMSE for each twin pair and each predictor
results <- twinpair %>%
  group_by(Twin) %>%
  summarise(across(predictor_cols, ~rmse(Age, .), .names = "RMSE_{.col}"))
  
# Print the results
print(results)

# Write to table
write.csv(results, paste(wkdr, '/Results/',GEOID,'_RMSE_Multi_Clock.tsv', sep=""),sep="\t", row.names=FALSE)

 [1] "mAge_BernabeuE2023c" "mAge_CBL_common"     "mAge_CBL_specific"  
 [4] "mAge_Cortex_common"  "mAge_epiTOC2"        "mAge_HannumG2013"   
 [7] "mAge_HorvathS2018"   "mAge_LevineM2018"    "mAge_LuA2019"       
[10] "mAge_LuA2023p1"      "mAge_LuA2023p2"      "mAge_LuA2023p3"     
[13] "mAge_McEwenL2019"    "mAge_PCHannumG2013"  "mAge_PCHorvathS2013"
[16] "mAge_PCHorvathS2018" "mAge_PCPhenoAge"     "mAge_ShirebyG2020"  
[19] "mAge_ZhangQ2019"     "mAge_ZhangY2017"    


# A tibble: 88 × 21
   Twin       RMSE_mAge_BernabeuE2…¹ RMSE_mAge_CBL_common RMSE_mAge_CBL_specific
   <chr>                       <dbl>                <dbl>                  <dbl>
 1 dizygotic…                  12.5                 14.7                    1.39
 2 dizygotic…                   7.53                 7.45                   8.00
 3 dizygotic…                   4.09                 4.00                  15.0 
 4 dizygotic…                   5.63                 5.48                   8.80
 5 dizygotic…                  10.9                 13.2                    4.26
 6 dizygotic…                  14.5                 10.5                    7.67
 7 dizygotic…                   8.51                11.8                    6.01
 8 dizygotic…                   5.13                 3.98                   6.71
 9 dizygotic…                  10.5                  6.13                   4.22
10 dizygotic…                  11.0                  7.62                   4.20
# ℹ 78 m

Warning message in write.csv(results, paste(wkdr, "/Results/", GEOID, "_RMSE_Multi_Clock.tsv", :
“attempt to set 'sep' ignored”


In [89]:
# # Create the data frame
# df <- data.frame(
#   Twin = c(rep("dizygotic_twinpair_1", 7), rep("monozygotic_twinpair_1", 7)),
#   RMSE_mAge_BernabeuE2023c = c(12.49570014, 7.53026079, 4.089717538, 5.634270515, 10.87384813, 14.45257348, 8.512067759, 9.887798715, 9.920981788, 10.59738579, 22.53718689, 17.35213473, 18.43159031, 17.78050347),
#   RMSE_mAge_CBL_common = c(14.73368136, 7.447542682, 4.001785133, 5.475945837, 13.15330669, 10.4646059, 11.81737162, 13.08483496, 4.149858078, 12.38246639, 15.52105091, 14.83128354, 13.83463529, 17.78164294),
#   RMSE_mAge_CBL_specific = c(1.392895288, 7.997214887, 14.95097032, 8.804791174, 4.25531477, 7.670750605, 6.007322273, 2.301247339, 7.082249325, 3.750189693, 16.74604118, 7.836493687, 5.512770383, 4.63890795)
# )

# #print(df)

# Reshape the data
df_long <- results %>%
  pivot_longer(cols = starts_with("RMSE"),
               names_to = "Predictor",
               values_to = "RMSE")

# Create a new variable for the type of twin pair
df_long$Twin_Type <- ifelse(grepl("dizygotic", df_long$Twin), "Dizygotic", "Monozygotic")

#Remove the string "RMSE_mAge_" from the Predictor column
df_long$Predictor <- gsub("RMSE_mAge_", "", df_long$Predictor)

#Remove the "epiTOC2" since this predictor seems to be so bad
df_long <- df_long[df_long$Predictor != "epiTOC2",]

# Print the data frame
print(df_long)


# Create the plot
p <- ggplot(df_long, aes(x = Predictor, y = RMSE, fill = Twin_Type)) +
  geom_boxplot() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
  labs(title = "Box Plot of RMSE by Predictor and Twin Type",
       x = "DNA Methylation Age Predictor",
       y = "RMSE Value",
       fill = "Twin Type")

# Save the plot to a file with a specified width and height
ggsave(paste("my_plot.png",sep=""), plot = p, width = 15, height = 10)



# A tibble: 1,672 × 4
   Twin                 Predictor       RMSE Twin_Type
   <chr>                <chr>          <dbl> <chr>    
 1 dizygotic_twinpair_1 BernabeuE2023c 12.5  Dizygotic
 2 dizygotic_twinpair_1 CBL_common     14.7  Dizygotic
 3 dizygotic_twinpair_1 CBL_specific    1.39 Dizygotic
 4 dizygotic_twinpair_1 Cortex_common  25.0  Dizygotic
 5 dizygotic_twinpair_1 HannumG2013     6.75 Dizygotic
 6 dizygotic_twinpair_1 HorvathS2018    2.75 Dizygotic
 7 dizygotic_twinpair_1 LevineM2018    29.9  Dizygotic
 8 dizygotic_twinpair_1 LuA2019        59.1  Dizygotic
 9 dizygotic_twinpair_1 LuA2023p1      24.5  Dizygotic
10 dizygotic_twinpair_1 LuA2023p2       8.71 Dizygotic
# ℹ 1,662 more rows
